In [52]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)
!ls -lt '/content/gdrive/My Drive/' 
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, MaxPooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import binary_accuracy, AUC
from sklearn.utils import shuffle
from keras.models import load_model

Mounted at /content/gdrive
total 6661079
drwx------ 2 root root       4096 Jan 14 07:25 'FYP NBs'
-rw------- 1 root root    3767216 Jan 13 09:10  BBH_Classification_Model_3.h5
-rw------- 1 root root        107 Jan 13 09:10  checkpoint
-rw------- 1 root root       3261 Jan 13 09:10  BBH_Classification_3.ckpt.index
-rw------- 1 root root    3701371 Jan 13 09:10  BBH_Classification_3.ckpt.data-00000-of-00001
-rw------- 1 root root       3261 Jan 12 13:14  BBH_Classification_2.ckpt.index
-rw------- 1 root root    3701511 Jan 12 13:14  BBH_Classification_2.ckpt.data-00000-of-00001
-rw------- 1 root root    3766752 Jan 12 13:14  BBH_Classification_Model_2.h5
-rw------- 1 root root   28299544 Jan 11 10:44  BBH_Classification_Model_1.h5
-rw------- 1 root root   28234311 Jan 11 10:44  BBH_Classification.ckpt.data-00000-of-00001
-rw------- 1 root root       3295 Jan 11 10:44  BBH_Classification.ckpt.index
drwx------ 2 root root       4096 Jan 11 10:08  Positive_Validation_DATA
-rw------- 1 root 

In [53]:
postestfoldername = '/content/gdrive/MyDrive/Positive_Test_DATA'
negtestfoldername = '/content/gdrive/MyDrive/Negative_Test_DATA'

# postive and negative prefixes
postestprefix = 111000000
negtestprefix = 101000000

#samples of pos and neg training data
postestsamples = 12288
negtestsamples = 4096


def initialise():
  global postestprefix
  global negtestprefix
  postestprefix = 111000000
  negtestprefix = 101000000


# number of batches
batch_looping_times = 64

# number of samples in each batch
test_samples = (postestsamples//batch_looping_times) +(negtestsamples//batch_looping_times)

print(test_samples)
test_y=np.concatenate((np.ones(((postestsamples//batch_looping_times), 1)), np.zeros(((negtestsamples//batch_looping_times), 1))))
print(test_y.shape)

256
(256, 1)


In [54]:
def batchload_test():
  global test_y
  global postestprefix
  global negtestprefix
  # print(samples)
  X=np.zeros((test_samples,8192*2))
  # y=np.zeros((samples,1))
  for i in range(postestsamples//batch_looping_times):
    posfile_name = postestfoldername+'/'+str(postestprefix+i+1)+'.npy'
    X[i] = np.load(posfile_name)
    # y[i] = 1
  for i in range(negtestsamples//batch_looping_times):
    negfile_name = negtestfoldername+'/'+str(negtestprefix+i+1)+'.npy'
    X[i+ (postestsamples//batch_looping_times)] = np.load(negfile_name)
    # y[i] = 0

  X = X.reshape(test_samples, 16384,1)
  postestprefix += (postestsamples//batch_looping_times)
  negtestprefix += (negtestsamples//batch_looping_times)

  if (postestprefix>=12287):
    initialise()
  
  return X, test_y




In [55]:
from numpy import mean
from numpy import std
def summarize_results(scores):
	# print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))
 


In [47]:
model = load_model("/content/gdrive/MyDrive/BBH_Classification_Model_2.h5")
model.load_weights("/content/gdrive/My Drive/BBH_Classification_2.ckpt")
scores= np.zeros((64,1))
error_in_batch = np.zeros((64,1))
def evaluate():
  initialise()
  global model
  global scores
  global error_in_batch
  for turns in range(40):
    test_x, test_y = batchload_test()
    test_x, test_y = shuffle(test_x, test_y, random_state=54)
    score = model.evaluate(test_x, test_y, verbose = 0) 
    scores[turns] = score[1]*100
    error_in_batch[turns] = score[0]
  summarize_results(scores)

evaluate()

Accuracy: 60.303% (+/-46.710)


In [ ]:
print(scores)
print(error_in_batch)

In [56]:
test_samples = 16384
test_X=np.zeros((test_samples,8192*2))
test_y = np.zeros((test_samples,1))
test_y=np.concatenate((np.ones((postestsamples, 1)), np.zeros((negtestsamples, 1))))

for i in range(postestsamples):
  posfile_name = postestfoldername+'/'+str(postestprefix+i+1)+'.npy'
  test_X[i] = np.load(posfile_name)

for i in range(negtestsamples):
  negfile_name = negtestfoldername+'/'+str(negtestprefix+i+1)+'.npy'
  test_X[i+ postestsamples] = np.load(negfile_name)

test_X = test_X.reshape(test_samples, 16384,1)
test_X, test_y = shuffle(test_X, test_y, random_state=54)
model = load_model("/content/gdrive/MyDrive/BBH_Classification_Model_2.h5")
model.load_weights("/content/gdrive/My Drive/BBH_Classification_2.ckpt")
score = model.evaluate(test_X, test_y, verbose = 0) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

Test loss: 0.11091703921556473
Test accuracy: 0.96221923828125


In [57]:
test_X, test_y = shuffle(test_X, test_y, random_state=54)
model = load_model("/content/gdrive/MyDrive/BBH_Classification_Model_3.h5")
model.load_weights("/content/gdrive/My Drive/BBH_Classification_3.ckpt")
score = model.evaluate(test_X, test_y, verbose = 0) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

Test loss: 0.0993947684764862
Test accuracy: 0.96600341796875
